# LLM Mastery


## 1.Import all packages

In [ ]:
from datetime import datetime
from dotenv import load_dotenv
import io
import ipdb
import os
import pdb
import platform
import requests
import sentencepiece as spm
import shutil
import sys
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim
from tqdm import tqdm
import wandb
import zipfile

In [ ]:
# !pip install wandb

## 2.clear cache

In [ ]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

torch.cuda.empty_cache()

import gc
# del tensor_variable
gc.collect()

In [ ]:
# downlaod necessary files
# wiki.txt
# wiki_tokenizer.model
# wiki_tokenizer.vocab
# encoded_date.pt

# files_url = 'https://ideami.com/llm_train'
# print ('Downloading file')
# response = requests.get(files_url)
# zipfile.ZipFile(io.BytesIO(response.content)).extractall('.')


In [ ]:
# !pip install -q pdb

In [ ]:
# def calculate (v1, v2):
#     ipdb.set_trace()
#     # pdb.set_trace()
#     calc1 = v1 *55
#     calc2 = calc1 * v2
#     return calc2

# print (calculate (2, 3))

## Initialize parameters

In [2]:
# architecture parameters
batch_size = 8
context = 512
embed_size = 384
n_layers = 7
n_heads = 7
BIAS = True

# Hyperparameters
lr = 3e-4
dropout = 0.05
weight_decay = 0.01
grad_clip = 1.0

# Training parmters
train_iters = 100000
eval_interval = 50
eval_iters = 10
compile = False
load_pretrained = False
checkpoint_dir = 'models/'
# checkpoint_fn = 'llm2.pt'
# checkpoint_load_fn = 'llm2.pt'
checkpoint_fn = 'latest.pt'
checkpoint_load_fn = 'latest.pt'
dtype = torch.bfloat16

# Mode
inference = False

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print ('device you will be isueing:', device)

device you will be isueing: cuda


## Set up logging
does not work yet

In [3]:
import wandb
from datetime import datetime
# logging
    # 2a4ae038692c0e045e311440fb0bd84bda2c7a
load_dotenv()
wandb_api_key = os.getenv('WANDB_API_KEY')
wandb_log = True
wandb_project = 'llm9'
# wandb_run_name = 'llm9-'+datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
wandb_run_name = 'llm9run'
# 

wandb.login(key=wandb_api_key)
if wandb_log:
    import wandb
    wandb.init(project=wandb_project, name=wandb_run_name)

    

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Tiger\_netrc
wandb: Currently logged in as: hassanalam (hassanalam-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Read Text

In [ ]:
# some code
with open('wiki.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

print (text[30000:30300])


### Set up tokenizer

In [5]:
# tokensizer
sp = spm.SentencePieceProcessor(model_file= 'wiki_tokenizer.model')
vocab_size = sp.get_piece_size()
print (f'Tokenizer vocab_size:  {vocab_size}')

Tokenizer vocab_size:  4096


## Set up encoder decoder

In [6]:
# encoder decoder
encode = lambda s: sp.Encode(s)
decode = lambda l: sp.Decode(l)
print (encode ('once upon a tine'))
print (decode (encode ('once upon a time')))

[2686, 698, 265, 261, 259, 500]
once upon a time


## load encoded data

In [7]:
if os.path.exists(f'encoded_data.pt'):
    data = torch.load('encoded_data.pt')
else:
    data = torch.tensor(encode(text), dtype=torch.long)
    torch.save(data,'encode_data.pt')

## Split into train validation

In [8]:
data_size = len(data)
spl = int(0.9*data_size)
train_data = data[:spl]
val_data = data[spl:]

print (f'train data: {len(train_data)/1e6:2f} Million : validate data: {len(val_data)/1e6:2f}')

train data: 53.289969 Million : validate data: 5.921108


## Set up batch

In [9]:
# get batch
def get_batch(split):
    # BS = BatchSize XL = Sequence or context length
    data = train_data if split == 'train' else val_data
    inds = torch.randint(len(data)+context, (batch_size, ) )
    x = torch.stack([data[i: i+context] for i in inds]) # (BS, SL)
    y = torch.stack([data[i+1: i+context +1] for i in inds])

    x,y = x.to(device), y.to(device)
    return x,y


In [32]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    # Only select indices that allow for full context-length sequences
    max_index = len(data) - context
    inds = torch.randint(0, max_index, (batch_size,))
    x = torch.stack([data[i: i+context] for i in inds])
    y = torch.stack([data[i+1: i+context+1] for i in inds])
    return x.to(device), y.to(device)

In [ ]:
# x, y = get_batch('train')
# print (x.shape, y.shape)
# print (x[0][:10])
# print (y[0][:10])

# # Find these lines in your GPT class
# torch.nn.init.normal(module.weight, mean = 0.0, std = 0.02)
# torch.nn.init.normal(module.weight, mean= 0.00, std=0.02)

# # And replace with
# torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
# torch.nn.init.normal_(module.weight, mean= 0.00, std=0.02)

## Coding LLM Architecture
19 Million parameter LLM
With 8 Batch Size 
with 128 batch Size will require 24 GB
Adjust Batch Size
Because of small data set and model results will be limite
demonstrate improvement during train


In [22]:
class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_size)
        self.positions = nn.Embedding(context, embed_size)
        self.blocks = nn.Sequential(*[Block (n_heads) for _ in range (n_layers)] )
        self.ln = nn.LayerNorm(embed_size)
        self.final_linear = nn.Linear(embed_size, vocab_size, bias=BIAS)
        self.apply(self._init_weights)

    # parameter initializtion
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean= 0.00, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)

    def forward(self, input, targets = None):
        loss = None
        BS, SL = input.shape
        emb = self.embeddings(input) #BS x XL x 284
        pos = self.positions(torch.arange(SL, device=device)) # SL x 384
        x = emb + pos
        x = self.blocks(x) # BS x SL x 384
        x = self.ln(x) # BS x SL x 384
        logits = self.final_linear(x) # sclaed number

        if targets is not None:
            BS, SL, VS = logits.shape
            logits = logits.view (BS*SL, VS)
            targets = targets.view(BS*SL)
            loss = F.cross_entropy(logits, targets)

            # manual calcualtion
            counts = logits.exp()
            prob = counts / counts.sum(1, keepdim=True)
            loss2 = - prob[torch.arange(BS*SL), targets].log().mean()

            if (not torch.allclose(loss, loss2)):
                print (f'(Loss diff) Pytorch: {loss.item()} manual: {loss2.item()}')

        return logits, loss
    
    def generate(self, input, max = 500):
        for _ in range (max):
            input = input[:,-context:] # (1, input lenght unte max of SL)
            logits, _ = self (input) # resutl (1, input lenght, 4096)
            logits = logits [:,-1,: ] # pick last probablity
            probs = F.softmax(logits, dim = -1)# # (1, 4096)
            next = torch.multinomial(probs, num_samples = 1)
            input = torch.cat((input, next), dim = 1)
        return input



In [11]:
class Block (nn.Module):
    def __init__(self, n_heads):
        super().__init__()
        head_size = embed_size // n_heads
        self.ma = Multihead(n_heads, head_size)
        self.feed_forward = ForwardLayer(embed_size)
        self.ln1 = nn.LayerNorm(embed_size)
        self.ln2 = nn.LayerNorm(embed_size)

    def forward (self, x):
        x = x + self.ma(self.ln1(x))
        x = x + self.feed_forward(self.ln2(x))
        return x





In [12]:
class ForwardLayer(nn.Module):
    def __init__(self, embed_size):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(embed_size,6*embed_size, bias= BIAS),
            nn.GELU(),
            nn.Linear(6*embed_size, embed_size, bias = BIAS),
            nn.Dropout(dropout)
    )
    def forward (self, x):
        x = self.network(x)
        return (x)
    

In [13]:
class Multihead(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        self.combine = nn.Linear(head_size * n_heads, embed_size, bias = BIAS)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x = torch.cat([Head(x) for head in self.heads], dim = 1)
        x = torch.cat([head(x) for head in self.heads], dim = -1)        # each  head ouputs (BS, SL, head_size)
        x = self.combine(x)
        x = self.dropout(x)
        return x 

In [15]:
# Head attention layer
# Detects and reinforces patterns in relationship memembers of sequesnce
class Head(nn.Module):
    def __init__ (self, head_size):
        super().__init__()
        self.queries = nn.Linear(embed_size, head_size, bias=BIAS)
        self.keys = nn.Linear(embed_size, head_size, bias=BIAS)
        self.values = nn.Linear(embed_size, head_size, bias=BIAS)

        self.register_buffer('tril', torch.tril(torch.ones(context,context)))
        self.dropout = nn.Dropout(dropout)

    def forward (self,x):
        BS, SL, VS = x.shape
        q=self.queries(x) #BS, SL, 54
        k=self.keys(x)
        v=self.values(x)

        attn_w = q @ k.transpose(-2, -1) * k.shape [-1]**-0.5 # BS, SL, SL
        attn_w = attn_w.masked_fill(self.tril[:SL, :SL]== 0, float ('-inf'))
        attn_w = F.softmax(attn_w, dim = -1) #BS, SL, SL

        x = attn_w @ v #BS, SL, SL
        return x


In [ ]:
# # understand attention
# x,y = get_batch('train')
# print (f'shape of x, y {x.shape, y.shape}')
# x= x.to(device)
# y= y.to(device)
# head_size = embed_size // n_heads
# embeddings = nn.Embedding(vocab_size, embed_size).to(device)
# positions = nn.Embedding(context, embed_size).to(device)
# print(f'positions {positions}')
# queries = nn.Linear(embed_size, head_size, bias=BIAS).to(device)
# keys = nn.Linear(embed_size, head_size, bias=BIAS).to(device)
# values = nn.Linear(embed_size, head_size, bias=BIAS).to(device)
# tril = torch.tril(torch.ones(context,context)).to(device)

# emb = embeddings(x)
# # pos = positions(torch.arange(context, device =device))
# # print (f'pos {pos}')
# x = emb + pos

# q=queries(x)
# k=keys(x)
# v=values(x)
# print(f'shape of q, k, v {q.shape, k.shape, v.shape}')
# torch.set_printoptions(precision=2, sci_mode=False)
# print(q[0][0])

# attn_w = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
# # print (f'attn_w 1: {attn_w[:, :, 7]}')
# attn_w = attn_w.masked_fill(tril[:context, :context]==0, float('-inf'))
# # print (f'attn_w 2: {attn_w[:, :, 7]}')
# attn_w = F.softmax(attn_w, dim = -1)
# # print (f'attn_w 3: {attn_w[:, :, 7]}')
# x = attn_w @ v

In [ ]:
# # understand attention matrix
# full = q @ k.transpose(-2, -1)
# a = q[0][5]
# b = k.transpose(-2, -1)[0,:,3]
# print (f'a, b: {a, b}')
# c = torch.dot (a, b)
# torch.set_printoptions(precision=2, sci_mode=False)
# print (f'c: {c:.2f}')
# print (f'full 0, 5, 3: {full[0][5][3]:.2f}')

In [ ]:
# # underand th eupadtin of the V content
# # print(f'attn_w: {attn_w[:,:,7]} \nv: {v[:,:,7]}')
# print (f'attn_w and v shape: {attn_w.shape, v.shape}')

# print (f'x 07: {x[0][7]}')

# attn_scores2 = attn_w[0, 7, :] # Shape [512]
# # print (attn_scores2)
# result = torch.zeros (54)
# for i in range (54):
#     result[i] = torch.dot (attn_scores2, v[0, :, i])

# print (f'result: {result}')

In [ ]:
# # test code
# x, y = get_batch('train')
# print (x.shape, y.shape)
# print (x[0][:10])
# print (y[0][:10])
# model = GPT()
# model = model.to(dtype = dtype)
# model = model.to(device=device)


# logits, loss = model (x, y)
# print (f'loss: {loss.item()}')
# # logits, loss, loss2 = model (x, y)
# # # print (loss.item(), loss2.item())

In [23]:
# Training Setup
model = GPT()
model = model.to(dtype)
model = model.to(device)

if compile:
    print ('Torch :: Compiling model')
    model = torch.compile(model)


sum(p.numel() for p in model.parameters())/1e6

19.837954

In [24]:
#  Caluclate loss averages
@torch.no_grad()
def calcuate_loss():
    out = {}
    model.eval()
    for split in ['train', 'eval']:
        l = torch.zeros (eval_iters)
        for i in range (eval_iters):
            x, y = get_batch(split)
            _, loss = model(x, y)
            l[i] = loss
        out[split]=l.mean().item()
    model.train
    return out

# l = calcuate_loss()
# print (l)

In [25]:
# Setting up the optimizer
import torch.optim

p_dict = {p_name: p for p_name, p in model.named_parameters() if p.requires_grad}

weight_decay_p = [p for n, p in p_dict.items() if p.dim() >= 2]

no_weight_decay_p = [p for n, p in p_dict.items() if p.dim() < 2]

optimizer_groups =[ 
    {'params':weight_decay_p, 'weight_decay':weight_decay},
    {'params':no_weight_decay_p, 'weight_decay': 0.0}

]


optimizer = torch.optim.Adam(optimizer_groups , lr = lr, betas = (0.9, 0.99))

scheduler =  torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, train_iters, eta_min=lr/10)

start_interation  = 0
best_val_loss = float('inf')



In [26]:
# Loading Checkpoints
def load_checkpoint(path):
    print ('llm = loading model')
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    iteration = checkpoint['iteration']
    loss = checkpoint['loss']
    print (f'load for iter {iteration}')
    return iteration, loss

if os.path.exists(f'{checkpoint_dir}/{checkpoint_fn}') and load_pretrained:
    start_iteration, loss=load_checkpoint(checkpoint_dir + checkpoint_fn)
    best_val_loss = loss





In [19]:
@torch.no_grad()
def generate_sample(input):
    t1 = torch.tensor(encode (input), dtype = torch.long, device = device)
    t1 = t1[None, :] # (1, (ssixe of the ids))
    newgen = model.generate(t1, max = 64)[0].tolist()
    result = decode (newgen)
    print (f'{result}')

# generate_sample('once upon a time')

In [27]:
from ipywidgets import widgets
from IPython.display import display

# Inference
if inference == True:
    model.eval()
    
    # Create the text input with styled parameters
    text_input = widgets.Text(
        description='Enter text:',
        placeholder='Type here and press Enter',
        description_tooltip='Enter your prompt text here',
        layout=widgets.Layout(
            width='500px',
            height='40px'
        ),
        style={
            'description_width': '100px',
            'font_family': 'monospace',
            'font_size': '16px'
        }
    )
    
    output = widgets.Output()
    
    def on_submit(sender):
        with output:
            output.clear_output()
            qs = text_input.value
            
            # Clear the input box
            text_input.value = ''
            
            if qs == 'q':
                print("Quitting...")
                return
            
            if qs == '':
                print("Please enter some text")
                return
                
            print(f"Processing: {qs}")
            # Call your generate_sample function with the input
            generate_sample(qs)
    
    text_input.on_submit(on_submit)
    display(text_input, output)

In [ ]:
# training loop
try:
    for i in tqdm (range (start_interation, train_iters)):
        xb, yb = get_batch ('train')
        logits, loss = model (xb, yb)

        if(i % eval_interval == 0 or i == train_iters -1):
            l = calcuate_loss()
            print (f'\n{i}: train loss: {l["train"]}/ val los: {l["eval"]}')
            generate_sample ('Once upon a time')

            if l['eval'] < best_val_loss:
                best_val_loss = l['eval']
                torch.save({
                    'model_state':model.state_dict(),
                    'optimizer_state_dict':optimizer.state_dict(),
                    'loss':best_val_loss,
                    'iteration': i,

                }, checkpoint_dir + checkpoint_fn)

            if wandb_log:
                    wandb.log({
                    'loss/train': l['train'],
                    'loss/val': l['eval'],
                    'lr': scheduler.get_last_lr()[0], 
                    },
                    step = i)   

        optimizer.zero_grad(set_to_none=True)
        loss.backward()

        nn.utils.clip_grad_norm(model.parameters(),max_norm=grad_clip)

        optimizer.step()
        scheduler.step()

    if wandb_log:
        wandb.finish()

except KeyboardInterrupt:
    print ('Training interuupted, Cleaning up')

finally:
    torch.cuda.empty_cache()
    sys.exit()




  0%|          | 0/100000 [00:00<?, ?it/s]


0: train loss: 5.512499809265137/ val los: 5.481249809265137


C:\Users\Tiger\AppData\Local\Temp\ipykernel_51436\209142334.py:33: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(model.parameters(),max_norm=grad_clip)
  0%|          | 2/100000 [00:03<37:27:29,  1.35s/it]

Once upon a time.The provincetars� rather Guramomism is the ter parts of orig Conn angment and Sp Games off Recim finaling head. This was be is a Ria's. The human'sus was ang- Carolina, M:of� wrestft flag– differer


  0%|          | 50/100000 [00:09<3:24:18,  8.15it/s]

...

  3%|▎         | 2900/100000 [08:27<2:53:41,  9.32it/s] 


2900: train loss: 5.518750190734863/ val los: 5.5625


  3%|▎         | 2902/100000 [08:31<18:13:40,  1.48it/s]

Once upon a time. The m established reached be foc e Mac lik someone northeastoins " particant particip is causeoviemhinlex. originallyxming in the pass about 12, on in the pl Compevaterummer. M4ational 28 Atale. It had a spiror of the firstrew arch


  3%|▎         | 2948/100000 [08:36<3:01:19,  8.92it/s] 

In [31]:
!nvidia-smi

Tue Mar 25 19:00:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.36                 Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080      WDDM  |   00000000:01:00.0  On |                  N/A |
| 53%   37C    P5             60W /  370W |    8380MiB /  10240MiB |     37%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----


|    0   N/A  N/A     19200    C+G   ...l\Microsoft\Teams\current\Teams.exe      N/A      |
|    0   N/A  N/A     19328    C+G   ...a\Local\slack\app-4.43.43\slack.exe      N/A      |
|    0   N/A  N/A     19992    C+G   ...4.0_x64__cv1g1gvanyjgm\WhatsApp.exe      N/A      |
|    0   N/A  N/A     20652    C+G   ..._x64__kzf8qxf38zg5c\Skype\Skype.exe      N/A      |
|    0   N/A  N/A     21320    C+G   ...m Files\Mozilla Firefox\firefox.exe      N/A      |
|    0   N/A  N/A     21784    C+G   ...hSmith\Snagit 2023\SnagitEditor.exe      N/A      |
|    0   N/A  N/A     23940    C+G   ...siveControlPanel\SystemSettings.exe      N/A      |
|    0   N/A  N/A     25228    C+G   ...on\134.0.3124.72\msedgewebview2.exe      N/A      |
|    0   N/A  N/A     28456    C+G   ...5n1h2txyewy\ShellExperienceHost.exe      N/A      |
|    0   N/A  N/A     29816    C+G   ...a\Local\slack\app-4.43.43\slack.exe      N/A      |
|    0   N/A  N/A     34296    C+G   ...m Files\Mozilla Firefox\firefox.exe    